### DOCUMENTATION
This notebook is intended to setup a PostgreSQL table with the data from the source csv-file.

#### INPUT
Use the "settings.ini"-file to set your database credentials, as well as connection parameters, the table name and more. 
This file gets parsed as configuration for the connection to the database as well as the definition of parameters.

#### PREPARATION
Make sure you have a running installation of [PostgreSQL](https://www.postgresql.org/download/) and have created a database via either the CLI or a GUI (e.g. [DBeaver](https://dbeaver.io/), [pgAdmin4](https://www.pgadmin.org/download/))
Also, make sure to install the [PostGIS](https://postgis.net/documentation/getting_started/#installing-postgis) extension for PostgreSQL.

#### WORKFLOW
The following is a description of steps which the notebook works through.
- the notebook creates an connection with the exisiting PostgreSQL database
- if not done already, the PostGIS extension will be activated in PostgreSQL
- the source csv-file is read and as a geopands dataframe, with the latitude and longitude columns being converted into a PostGIS compatible geometric point data type
- NULL are set correctly and the columns are renamed in a consistent format
- finally the cleaned geopandas dataframe is used to create a table inside the PostgreSQL database

Besides the INPUT and PREPARATION step no user action is required.

In [1]:
import geopandas as gpd
from sqlalchemy import create_engine, text, SmallInteger, Integer
from sqlalchemy.exc import ProgrammingError
import numpy as np
from configparser import ConfigParser

In [2]:
config = ConfigParser()
config.read("database_settings.ini")
ini = config["default"]

In [3]:
engine = create_engine(f"postgresql://{ini['db_user']}:{ini['db_password']}@{ini['db_host']}:{ini['db_port']}/{ini['db_name']}")

In [4]:
conn = engine.connect()

In [5]:
try:
    conn.execute(text("CREATE EXTENSION postgis"))
    conn.execute(text("CREATE EXTENSION postgis_topology"))
    conn.commit()
except ProgrammingError as e:
    conn.rollback()
    print(e)


(psycopg2.errors.DuplicateObject) FEHLER:  Erweiterung »postgis« existiert bereits

[SQL: CREATE EXTENSION postgis]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [6]:
try:
    result = conn.execute(text("SELECT postgis_full_version()"))
    conn.commit()
    print(result.fetchall())
except ProgrammingError as e:
    conn.rollback()
    print(e)
    print("Error: PostGIS isn't installed correctly!")

[('POSTGIS="3.3.2 3.3.2" [EXTENSION] PGSQL="150" GEOS="3.11.1-CAPI-1.17.1" PROJ="7.2.1" LIBXML="2.9.9" LIBJSON="0.12" LIBPROTOBUF="1.2.1" WAGYU="0.5.0 (Internal)" TOPOLOGY',)]


In [7]:
dataset = gpd.GeoDataFrame.from_file(ini["path_to_csv"])
dataset = gpd.GeoDataFrame(dataset, geometry=gpd.points_from_xy(dataset.longitude, dataset.latitude), crs="EPSG:4326")

dataset = dataset.rename(columns={'geometry':'geom'}).set_geometry('geom')

In [8]:
dataset["Reviews"] = np.where(dataset['Reviews'].str.contains("nil", case=False), np.nan, dataset['Reviews'])

In [9]:
dataset.drop(["latitude", "longitude"], axis=1, inplace=True)

In [10]:
dataset.rename(inplace=True, columns={
    'Age': 'age',
    'Gender': 'gender',
    'Marital Status': 'marital_status',
    'Occupation': 'occupation',
    'Monthly Income': 'monthly_income',
    'Educational Qualifications': 'education',
    'Family size': 'family_size',
    'Pin code': 'pin_code',
    'Medium (P1)': 'medium_p1',
    'Medium (P2)': 'medium_p2',
    'Meal(P1)': 'meal_p1',
    'Meal(P2)': 'meal_p2',
    'Perference(P1)': 'preference_p1',
    'Perference(P2)': 'preference_p2',
    'Ease and convenient': 'ease_and_convenience',
    'Time saving': 'time_saving',
    'More restaurant choices': 'more_restaurant_choices',
    'Easy Payment option': 'easy_payment_option',
    'More Offers and Discount': 'more_offers_and_discounts',
    'Good Food quality': 'good_food_quality',
    'Good Tracking system': 'good_tracking_system',
    'Self Cooking': 'self_cooking',
    'Health Concern': 'health_concern',
    'Late Delivery': 'late_delivery',
    'Poor Hygiene': 'poor_hygiene',
    'Bad past experience': 'bad_past_experience',
    'Unavailability': 'unavailable',
    'Unaffordable': 'unaffordable',
    'Long delivery time': 'long_delivery_time',
    'Delay of delivery person getting assigned': 'delivery_assignment_delay',
    'Delay of delivery person picking up food': 'delivery_pickup_delay',
    'Wrong order delivered': 'wrong_order',
    'Missing item': 'missing_item',
    'Order placed by mistake': 'ordered_by_mistake',
    'Influence of time': 'time_influence',
    'Order Time': 'order_time',
    'Maximum wait time': 'max_wait_time',
    'Residence in busy location': 'busy_location_residence',
    'Google Maps Accuracy': 'google_maps_accuracy',
    'Good Road Condition': 'good_road_condition',
    'Low quantity low time': 'low_quantity_low_time',
    'Delivery person ability': 'deliver_person_availability',
    'Influence of rating': 'rating_influence',
    'Less Delivery time': 'less_delivery_time',
    'High Quality of package': 'high_package_quality',
    'Number of calls': 'call_number',
    'Politeness': 'politeness',
    'Freshness': 'freshness',
    'Temperature': 'temperature',
    'Good Taste': 'good_taste',
    'Good Quantity': 'good_quantity',
    'Output': 'output',
    'Reviews': 'reviews',
})

In [11]:
dataset.to_postgis(ini["table_name"], engine, index=True, if_exists="replace",  dtype={"age": SmallInteger(), "family_size": SmallInteger(), "pin_code": Integer()})

In [12]:
conn.close()